OSSとCRALWデータ、及び単漢字から訓練する

In [10]:
import os
with open('../version.txt','r',encoding='utf-8') as f:
    for l in f:
        ver=l.rstrip()
        break
os.environ["VER"]=ver
os.environ['DATA']='../dict/data/data.'+os.environ["VER"]
os.environ['DIR']='dataset/dataset.'+os.environ["VER"]
os.environ['LOG']='logs/logs.'+os.environ["VER"]
os.environ['MODEL']='model/model.'+os.environ["VER"]

In [11]:
!echo $VER

1.8o


In [12]:
#training

In [ ]:
!python ./transformer_model.py \
  --emb_size 512 \
  --nhead 8 \
  --ffn_hid_dim 2048 \
  --batch_size 64 \
  --num_encoder_layers 8 \
  --num_decoder_layers 8 \
  --lr 0.00002 \
  --dropout 0.3 \
  --num_epochs 100 \
  --device cuda \
  --earlystop_patient 3 \
  --output_dir $MODEL \
  --tensorboard_logdir $LOG \
  --train_file $DIR/train.jsonl \
  --valid_file $DIR/valid.jsonl


[00:00:11] Pre-processing sequences       ██████████████████ 0        /        0[00:00:00] Pre-processing sequences       ██████████████████ 0        /        0huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/analysis01/src/kanjikana-model/venv/lib/python3.10/site-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False becau

In [ ]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
%tensorboard --logdir logs/logs.$VER

Reusing TensorBoard on port 6006 (pid 4161637), started 0:02:15 ago. (Use '!kill 4161637' to kill it.)

In [ ]:
!python ./generate.py \
    --test_file $DIR/test.jsonl \
    --model_file $MODEL/checkpoint_best.pt \
    --outfile $DIR/generate.txt \
    --source_lang kana \
    --target_lang kanji \
    --device cpu \
    --nbest 5 \
    --beam_width 5 \
    --max_len 100 \
    --search beam

Traceback (most recent call last):
  File "/home/analysis01/src/kanjikana-model/train/./generate.py", line 191, in <module>
    main()
  File "/home/analysis01/src/kanjikana-model/train/./generate.py", line 187, in main
    KanjiKanaTransformerTest(args).generate()
  File "/home/analysis01/src/kanjikana-model/train/./generate.py", line 131, in generate
    last_checkpoint = torch.load(self.args.model_file, map_location=torch.device(self.args.device))
  File "/home/analysis01/src/kanjikana-model/venv/lib/python3.10/site-packages/torch/serialization.py", line 1484, in load
    with _open_file_like(f, "rb") as opened_file:
  File "/home/analysis01/src/kanjikana-model/venv/lib/python3.10/site-packages/torch/serialization.py", line 759, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/home/analysis01/src/kanjikana-model/venv/lib/python3.10/site-packages/torch/serialization.py", line 740, in __init__
    super().__init__(open(name, mode))
FileNotFoundError: [Errno 2] N

In [ ]:
!python evaluate.py --infile $DIR/generate.txt

Traceback (most recent call last):
  File "/home/analysis01/src/kanjikana-model/train/evaluate.py", line 60, in <module>
    main()
  File "/home/analysis01/src/kanjikana-model/train/evaluate.py", line 56, in main
    run(args)
  File "/home/analysis01/src/kanjikana-model/train/evaluate.py", line 15, in run
    with open(args.infile,'r',encoding='utf-8') as f:
FileNotFoundError: [Errno 2] No such file or directory: 'dataset/dataset.1.8o/generate.txt'


In [ ]:
!date

2025年  9月 29日 月曜日 15:12:16 JST
